In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn import metrics
import os
import time

In [2]:
def DBSCAN_sklearn(X,eps,min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    return labels,n_clusters_,n_noise_

In [3]:
ORIGINAL_PATH = os.path.join('.', 'MOCK_DATA (1).xlsx')

global df

df = pd.read_excel(ORIGINAL_PATH)

print(df)

                             Project             Country       Start  \
0    Kohler, Wilkinson and Buckridge  Dominican Republic   9/20/2021   
1                   Smitham and Sons               China    8/3/2020   
2          Jenkins, Huels and Zulauf       United States    1/5/2023   
3          Terry, Emmerich and Berge              Sweden   12/7/2021   
4      Padberg, Schamberger and Will         Philippines   3/19/2023   
..                               ...                 ...         ...   
995  Sauer, Daugherty and McLaughlin               China  11/14/2019   
996                     Terry-Grimes               China   8/17/2023   
997                      Dickens LLC              France    9/7/2020   
998                       Hauck-Auer              Poland   4/23/2020   
999               Reynolds-Hettinger              Russia   1/16/2020   

            End        Budget  FindingSource (%)     Category  TRL Start  \
0     8/14/2024  $36399069.63               77.0      captu

In [4]:
drop_colomn =['Project','Country']
df.drop(columns=drop_colomn, inplace=True)

In [5]:
print(df)

          Start         End        Budget  FindingSource (%)     Category  \
0     9/20/2021   8/14/2024  $36399069.63               77.0      capture   
1      8/3/2020   4/18/2024  $48806994.79               84.0  utilization   
2      1/5/2023   6/10/2024  $11548939.83               82.6  utilization   
3     12/7/2021   12/4/2023  $17512435.77               71.1  utilization   
4     3/19/2023   8/18/2024  $18929345.53               66.8      capture   
..          ...         ...           ...                ...          ...   
995  11/14/2019   8/29/2024  $51230399.26               51.7   Full Chain   
996   8/17/2023   3/23/2026  $23925572.49               50.3          T&S   
997    9/7/2020    7/3/2025  $87475622.39               93.4          T&S   
998   4/23/2020  11/20/2024   $1259344.27               63.9  utilization   
999   1/16/2020   4/26/2026  $47882510.76               61.8   Full Chain   

     TRL Start  TRL End  profit (%)  Emissions decrease  
0            2   

In [6]:
df['Start'] = pd.to_datetime(df['Start'], format='%m/%d/%Y')
df['End'] = pd.to_datetime(df['End'], format='%m/%d/%Y')

In [7]:
df['Date_difference'] = (df['End'] - df['Start']).dt.days

In [8]:
print(df)

         Start        End        Budget  FindingSource (%)     Category  \
0   2021-09-20 2024-08-14  $36399069.63               77.0      capture   
1   2020-08-03 2024-04-18  $48806994.79               84.0  utilization   
2   2023-01-05 2024-06-10  $11548939.83               82.6  utilization   
3   2021-12-07 2023-12-04  $17512435.77               71.1  utilization   
4   2023-03-19 2024-08-18  $18929345.53               66.8      capture   
..         ...        ...           ...                ...          ...   
995 2019-11-14 2024-08-29  $51230399.26               51.7   Full Chain   
996 2023-08-17 2026-03-23  $23925572.49               50.3          T&S   
997 2020-09-07 2025-07-03  $87475622.39               93.4          T&S   
998 2020-04-23 2024-11-20   $1259344.27               63.9  utilization   
999 2020-01-16 2026-04-26  $47882510.76               61.8   Full Chain   

     TRL Start  TRL End  profit (%)  Emissions decrease  Date_difference  
0            2        3 

In [9]:
drop_colomn =['Start','End']
df.drop(columns=drop_colomn, inplace=True)

In [10]:
print(df)

           Budget  FindingSource (%)     Category  TRL Start  TRL End  \
0    $36399069.63               77.0      capture          2        3   
1    $48806994.79               84.0  utilization          7        5   
2    $11548939.83               82.6  utilization          3        4   
3    $17512435.77               71.1  utilization          6        3   
4    $18929345.53               66.8      capture          4        7   
..            ...                ...          ...        ...      ...   
995  $51230399.26               51.7   Full Chain          7        9   
996  $23925572.49               50.3          T&S          3        7   
997  $87475622.39               93.4          T&S          7        6   
998   $1259344.27               63.9  utilization          6        5   
999  $47882510.76               61.8   Full Chain          5        4   

     profit (%)  Emissions decrease  Date_difference  
0            -4                 6.3             1059  
1            

In [11]:
df.Category.unique()

array(['capture', 'utilization', 'T&S', 'Full Chain'], dtype=object)

In [12]:
df['Category'].replace(['utilization', 'capture', 'T&S', 'Full Chain'],[0,1,2,3],inplace=True)

In [13]:
print(df)

           Budget  FindingSource (%)  Category  TRL Start  TRL End  \
0    $36399069.63               77.0         1          2        3   
1    $48806994.79               84.0         0          7        5   
2    $11548939.83               82.6         0          3        4   
3    $17512435.77               71.1         0          6        3   
4    $18929345.53               66.8         1          4        7   
..            ...                ...       ...        ...      ...   
995  $51230399.26               51.7         3          7        9   
996  $23925572.49               50.3         2          3        7   
997  $87475622.39               93.4         2          7        6   
998   $1259344.27               63.9         0          6        5   
999  $47882510.76               61.8         3          5        4   

     profit (%)  Emissions decrease  Date_difference  
0            -4                 6.3             1059  
1            -6                 1.5             1

In [14]:
for index,e in enumerate(df['Budget']): 
    df['Budget'][index] = e[1:]
    

C:\Users\Timgad informatique\AppData\Local\Temp\ipykernel_4056\978855766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Budget'][index] = e[1:]


In [15]:
df['Budget'] = df['Budget'].astype(float)


In [16]:
print(df)

          Budget  FindingSource (%)  Category  TRL Start  TRL End  profit (%)  \
0    36399069.63               77.0         1          2        3          -4   
1    48806994.79               84.0         0          7        5          -6   
2    11548939.83               82.6         0          3        4          14   
3    17512435.77               71.1         0          6        3           7   
4    18929345.53               66.8         1          4        7           0   
..           ...                ...       ...        ...      ...         ...   
995  51230399.26               51.7         3          7        9          21   
996  23925572.49               50.3         2          3        7          25   
997  87475622.39               93.4         2          7        6          12   
998   1259344.27               63.9         0          6        5           4   
999  47882510.76               61.8         3          5        4          13   

     Emissions decrease  Da

In [17]:
def z_score():
    for x in df.columns :
        if(df[x].dtype.kind in 'iufcb' ):
            df[x] = (df[x] - df[x].mean()) / df[x].std()

In [18]:
z_score()

In [19]:
print(df)

       Budget  FindingSource (%)  Category  TRL Start   TRL End  profit (%)  \
0   -0.516692           0.129961 -0.481346  -1.207901 -1.026350   -1.284319   
1   -0.080750           0.610736 -1.384435   0.740327 -0.157298   -1.438703   
2   -1.389782           0.514581 -1.384435  -0.818256 -0.591824    0.105135   
3   -1.180259          -0.275265 -1.384435   0.350681 -1.026350   -0.435208   
4   -1.130477          -0.570598 -0.481346  -0.428610  0.711753   -0.975552   
..        ...                ...       ...        ...       ...         ...   
995  0.004395          -1.607700  1.324831   0.740327  1.580805    0.645479   
996 -0.954939          -1.703855  0.421742  -0.818256  0.711753    0.954247   
997  1.277842           1.256349  0.421742   0.740327  0.277228   -0.049248   
998 -1.751299          -0.769777 -1.384435   0.350681 -0.157298   -0.666784   
999 -0.113231          -0.914010  1.324831  -0.038965 -0.591824    0.027943   

     Emissions decrease  Date_difference  
0       

In [20]:
X = df.to_numpy()

In [21]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

In [22]:
def agglomerative_clustering_sklearn(X, n_clusters):
    clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(X)
    clusters = []
    for i in range(n_clusters):
        clusters.append([])
    for i in range(len(X)):
        clusters[clustering.labels_[i]].append(i)
    return clusters

In [23]:
i = 2
clusters = agglomerative_clustering_sklearn(X, i)
print("Execution ended ")
labels = np.zeros(len(X))
for i in range(len(clusters)):
    for j in range(len(clusters[i])):
        labels[clusters[i][j]] = i
print("number of clusters: ", len(clusters))
score = silhouette_score(X, labels, metric='euclidean')
print("Sillouette score: ", score)

Execution ended 
number of clusters:  2
Sillouette score:  0.0642937364843988


In [25]:
print(labels.shape)

(1000,)


In [26]:
print(X.shape)

(1000, 8)


In [27]:
new_df = pd.concat([df, pd.Series(labels, name='Label')], axis=1)

In [28]:
print(new_df)

       Budget  FindingSource (%)  Category  TRL Start   TRL End  profit (%)  \
0   -0.516692           0.129961 -0.481346  -1.207901 -1.026350   -1.284319   
1   -0.080750           0.610736 -1.384435   0.740327 -0.157298   -1.438703   
2   -1.389782           0.514581 -1.384435  -0.818256 -0.591824    0.105135   
3   -1.180259          -0.275265 -1.384435   0.350681 -1.026350   -0.435208   
4   -1.130477          -0.570598 -0.481346  -0.428610  0.711753   -0.975552   
..        ...                ...       ...        ...       ...         ...   
995  0.004395          -1.607700  1.324831   0.740327  1.580805    0.645479   
996 -0.954939          -1.703855  0.421742  -0.818256  0.711753    0.954247   
997  1.277842           1.256349  0.421742   0.740327  0.277228   -0.049248   
998 -1.751299          -0.769777 -1.384435   0.350681 -0.157298   -0.666784   
999 -0.113231          -0.914010  1.324831  -0.038965 -0.591824    0.027943   

     Emissions decrease  Date_difference  Label  
0

In [29]:
global df2

df2 = pd.read_excel(ORIGINAL_PATH)

print(df2)

                             Project             Country       Start  \
0    Kohler, Wilkinson and Buckridge  Dominican Republic   9/20/2021   
1                   Smitham and Sons               China    8/3/2020   
2          Jenkins, Huels and Zulauf       United States    1/5/2023   
3          Terry, Emmerich and Berge              Sweden   12/7/2021   
4      Padberg, Schamberger and Will         Philippines   3/19/2023   
..                               ...                 ...         ...   
995  Sauer, Daugherty and McLaughlin               China  11/14/2019   
996                     Terry-Grimes               China   8/17/2023   
997                      Dickens LLC              France    9/7/2020   
998                       Hauck-Auer              Poland   4/23/2020   
999               Reynolds-Hettinger              Russia   1/16/2020   

            End        Budget  FindingSource (%)     Category  TRL Start  \
0     8/14/2024  $36399069.63               77.0      captu

In [30]:
new_df2 = pd.concat([df2, pd.Series(labels, name='Label')], axis=1)

In [36]:
print(new_df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Project             1000 non-null   object 
 1   Country             1000 non-null   object 
 2   Start               1000 non-null   object 
 3   End                 1000 non-null   object 
 4   Budget              1000 non-null   object 
 5   FindingSource (%)   1000 non-null   float64
 6   Category            1000 non-null   object 
 7   TRL Start           1000 non-null   int64  
 8   TRL End             1000 non-null   int64  
 9   profit (%)          1000 non-null   int64  
 10  Emissions decrease  1000 non-null   float64
 11  Label               1000 non-null   int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 93.9+ KB
None


In [35]:
new_df2['Label'] = new_df2['Label'].astype('int64')

In [39]:
OUTPUT_FILE = os.path.join('.', 'output.xlsx')
new_df2.to_excel(OUTPUT_FILE,header = True)